In [1]:
from ModelPredictiveControl.ModelPredictiveControl import ModelPredictiveControl
from Simulation.ForwardKinematics import ForwardKinematics
import numpy as np
import scipy.sparse as sp
np.set_printoptions(precision=6, suppress=True, linewidth=1200, threshold=np.inf)
import time

fk = ForwardKinematics()

N = 1
Nu = 1
dof = 7
Ts = 0.2
A = np.zeros((dof,dof))
B = np.eye(dof)

print(A)
print(B)

mpc = ModelPredictiveControl(1, 2, N, Nu, dof)
dt_vector = mpc.computeDeltaTimeVector(Ts, N, 1, 1)
print(dt_vector)

x0 = np.ones(dof)
J = fk.jacobian(x0)
J2 = fk.jacobian(x0*1.2)

J_list = []
J2_list = []
for i in range(Nu):
    J_list.append(J)
    J2_list.append(J2)

u0 = np.zeros(dof)
joint_limits = np.ones(dof)*3.14
velocity_limits = np.ones(dof)*1.56
acceleration_limits = np.ones(dof)*10

Qx = np.eye(dof)*0.01
Qu = np.eye(dof)*0.1
Qs = np.eye(8)*100_000
Qdu = np.eye(dof)*0.5
Qs = np.eye(8)*100_000
Q = mpc.initializeHessian(Qx, Qu, Qs, Qdu)
print(Q)

Aeq = mpc.initializeConstraintMatrix(A, B, J_list, dt_vector)

cscAeq = sp.csc_matrix(Aeq)


Aeq2 = mpc.updateConstraintMatrix(Aeq, J2_list)
cscAeq2 = sp.csc_matrix(Aeq2)

lower_constraint, upper_constraint = mpc.initializeConstraintVectors(Ts, x0, A, u0, joint_limits, velocity_limits, acceleration_limits)

x0 = np.ones(dof)*0.5
u0 = np.ones(dof)*0
ref = np.ones(8)*5
ref_list = [ref]*Nu

lower_constraint, upper_constraint = mpc.updateConstraintVector(lower_constraint, upper_constraint, Ts, x0, u0, ref_list)
print(lower_constraint)
print(upper_constraint)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
[0.2, 0.2]
  (0, 0)	0.01
  (1, 1)	0.01
  (2, 2)	0.01
  (3, 3)	0.01
  (4, 4)	0.01
  (5, 5)	0.01
  (6, 6)	0.01
  (7, 7)	0.1
  (8, 8)	0.1
  (9, 9)	0.1
  (10, 10)	0.1
  (11, 11)	0.1
  (12, 12)	0.1
  (13, 13)	0.1
  (14, 14)	100000.0
  (15, 15)	100000.0
  (16, 16)	100000.0
  (17, 17)	100000.0
  (18, 18)	100000.0
  (19, 19)	100000.0
  (20, 20)	100000.0
  (21, 21)	100000.0
  (22, 22)	0.5
  (23, 23)	0.5
  (24, 24)	0.5
  (25, 25)	0.5
  (26, 26)	0.5
  (27, 27)	0.5
  (28, 28)	0.5
[ -0.5   -0.5   -0.5   -0.5   -0.5   -0.5   -0.5    5.     5.     5.     5.     5.     5.     5.     5.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -3.14  -3.14  -3.14  -3.14  -3.14

In [2]:
test_array = np.array([[1, 2, 3, 0],[0, 1, 0, 0], [1,0,0,0], [1,0,0,0]])
csc = sp.csc_matrix(test_array)
print(test_array)
print(csc)
csc[0,1] = 0
print(csc)

[[1 2 3 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]]
  (0, 0)	1
  (2, 0)	1
  (3, 0)	1
  (0, 1)	2
  (1, 1)	1
  (0, 2)	3
  (0, 0)	1
  (2, 0)	1
  (3, 0)	1
  (0, 1)	0
  (1, 1)	1
  (0, 2)	3
